# Extração da Drogaria Minas Mais

In [79]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json
import pandas as pd
import math

In [80]:
# Capturando todas as URLs

# Função de acessar site
all_workers = os.cpu_count()

def access_site(url, max_attempts=all_workers):
    headers = {'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for retry in range(max_attempts):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
        except Exception as e:
                print(f"An error occurred while accessing URL: {url}: {e}. Retrying... (Attempt {retry + 1}/{max_attempts})")
                time.sleep(1)

# Função captura de todos os nomes de marcas e criação de URLs
def getNomeMarcas():
    response = access_site("https://www.drogariasminasmais.com.br/medicamentos")
    soup = BeautifulSoup(response.text, 'html.parser')
    sessaoMarcasGrandes = soup.find('div', class_='vtex-search-result-3-x-filter__container vtex-search-result-3-x-filter__container--responsive-result-content-mz0001 bb b--muted-4 vtex-search-result-3-x-filter__container--brand')
    sessaoMarcas = sessaoMarcasGrandes.find('div', class_='vtex-search-result-3-x-filterTemplateOverflow vtex-search-result-3-x-filterTemplateOverflow--responsive-result-content-mz0001 pb5 overflow-y-auto')
    marcas = sessaoMarcas.find_all('label', class_='vtex-checkbox__label w-100 c-on-base pointer')
    return marcas

# Criando uma lista de URLs ainda não bem estruturadas
urlSuja = getNomeMarcas()
urlLimpa = []

# Limpando lista das URLs
for i in urlSuja:
    # Criando texto separado e minusculo com "-"
    texto = i.text.lower()
    textoNovo = texto.replace(" ", "-")
    textoNovissimo = textoNovo.replace("/", "-")

    # Salvando tudo em uma lista
    urlLimpa.append(textoNovissimo)

listaUrls = []

# Criando todas as URLs
for j in urlLimpa:
    URL = "https://www.drogariasminasmais.com.br/medicamentos/" + j + "?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page="
    listaUrls.append(URL)

In [82]:
# Achando número de produtos
url = "https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1"
requisicao = requests.get(url)
soup = BeautifulSoup(requisicao.text, 'html.parser')
numeroProdutos = soup.find_all('script')[-34].text
numeroProdutos = numeroProdutos[:-8].split("quantity")[66:]

listaNumeroProdutos = []
listaNumeroProdutosCorreta = []

for i in numeroProdutos:
    listaNumeroProdutos.append(i.split(",")[0])

for i in listaNumeroProdutos:
    aux = i.replace('"', "")
    aux = aux.replace(":", "")
    listaNumeroProdutosCorreta.append(int(aux))

In [67]:
def getJSON(soup):
    jsonSujo = soup.find_all('script')[-34].text
    jsonSujo = jsonSujo[:-8]
    jsonLimpo = jsonSujo.split("__STATE__ = ")[1]
    return jsonLimpo

total_pages = []

for i in listaNumeroProdutosCorreta:
    total_pages.append(math.ceil(i//15))

listaJSON = []

for i in listaUrls: 
    for page_number in total_pages:
        for paginaAtual in range(1, page_number):
            url = str(i) + str(paginaAtual)
            requisicao = requests.get(url)
            soup = BeautifulSoup(requisicao.text, 'html.parser')
            listaJSON.append(getJSON(soup))

ConnectionError: HTTPSConnectionPool(host='www.drogariasminasmais.com.br', port=443): Max retries exceeded with url: /medicamentos/allergan?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=5 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fba88cd4100>: Failed to resolve 'www.drogariasminasmais.com.br' ([Errno -3] Temporary failure in name resolution)"))

In [91]:
def getJSON(soup):
    jsonSujo = soup.find_all('script')[-34].text
    jsonSujo = jsonSujo[:-8]
    jsonLimpo = jsonSujo.split("__STATE__ = ")[1]
    return jsonLimpo

total_pages = [25]
listaJSON = []

for page_number in total_pages:
    for paginaAtual in range(1, page_number + 1):
        url = "https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=" + str(paginaAtual)
        requisicao = requests.get(url)
        soup = BeautifulSoup(requisicao.text, 'html.parser')
        listaJSON.append(getJSON(soup))

In [93]:
# Criando o JSON e capturando todos os códigos de produtos
ean = []
precoComDesconto = []
precoSemDesconto = []
desconto = []
marcas = []


for i in listaJSON:
    dados = json.loads(listaJSON[1])
    listaProdutos = []
    listaProdutosCerta = []
    listaProdutosSuperCerta = []
    j = 0 

    for i in dados.keys():
        if j % 18 == 0:
            listaProdutos.append(i)
        j = j + 1
        if j == 270:
            break

    for i in listaProdutos:
        listaProdutosCerta.append(i.split(".")[0])

    for i in listaProdutosCerta:
        listaProdutosSuperCerta.append(i.replace("$", ""))

    # Pegando todos os dados dos produtos
    for i in listaProdutosSuperCerta:
        marcas.append(dados[i]["brand"])

    for i in listaProdutosSuperCerta:
        # Parte do JSON que tem o EAN e precos
        localJson1 = str(i) + '.items({"filter":"ALL"}).0'
        dadosDesejados1 = dados.get(localJson1)

        ean.append(dadosDesejados1["ean"])

    for i in listaProdutosSuperCerta:
        # Parte do JSON que tem o EAN e precos
        localJson2 = "$" + str(i) + '.items({"filter":"ALL"}).0.sellers.0.commertialOffer'
        dadosDesejados2 = dados.get(localJson2)

        precoComDesconto.append(dadosDesejados2["Price"])
        precoSemDesconto.append(dadosDesejados2["ListPrice"])
        desconto.append((1 - (dadosDesejados2["Price"]/dadosDesejados2["ListPrice"])))

In [100]:
data = {"marcas": marcas, "EAN": ean, "Preço com desconto": precoComDesconto, "Preço sem desconto": precoSemDesconto, "Desconto": desconto}
df = pd.DataFrame(data)

# Tratar df para não ter repetição
for i in df["EAN"]:
    for j in df["EAN"]:
        if i == j:
            n = i.index()
            df = df.drop(index=n, axis=1)

TypeError: index() takes at least 1 argument (0 given)

In [53]:
# Exportando tudo para um csv
df.to_excel('DadosMinasMais.xlsx',sheet_name='Sheet1')